In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x393a8000 @  0x7ff2861cb615 0x591f47 0x4cc229 0x4cc38b 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x509918 0x50a64d
     |████████████████████████████████| 846.0MB 21kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 2.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588513 sha256=cdf8739d8f900528f2a9683d6f08dd6eb5bfb2a6f77f56dbe77624108502dd89
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 2.7MB

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-vr_7ga3j
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-vr_7ga3j
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12825 sha256=02b32ffb21dfc308b5cb59a456df2067e7e0b7cb80551e2fb5b17c2dd3c0cda3
  Stored in directory: /tmp/pip-ephem-wheel-cache-0vn3zl0n/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [4]:
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

## 데이터 준비

In [7]:
##GPU 사용 시
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [11]:
angry = pd.read_excel("/content/drive/My Drive/Colab Notebooks/4-2고급기계플젝/dataset/angry.xlsx")
happy = pd.read_excel("/content/drive/My Drive/Colab Notebooks/4-2고급기계플젝/dataset/happy.xlsx")
love = pd.read_excel("/content/drive/My Drive/Colab Notebooks/4-2고급기계플젝/dataset/love.xlsx")
sad = pd.read_excel("/content/drive/My Drive/Colab Notebooks/4-2고급기계플젝/dataset/sad.xlsx")
depressed = pd.read_excel("/content/drive/My Drive/Colab Notebooks/4-2고급기계플젝/dataset/depressed.xlsx")

In [15]:
angry.columns=['sentence','label']
happy.columns=['sentence','label']
love.columns=['sentence','label']
sad.columns=['sentence','label']
depressed.columns=['sentence','label']

In [13]:
angry = angry.dropna()
happy = happy.dropna()
love = love.dropna()
sad = sad.dropna()
depressed = depressed.dropna()

In [13]:
data=pd.concat([angry,happy,love,sad,depressed])
data = data.reset_index(drop=True)

In [14]:
data.head()

,sentence,label
0,화나다와 화내다,4
1,여러분물어볼게있는데요.화나다하고속상...,4
2,화나다와 화내다의 구분방법,4
3,‘화나다’는 동사인가요 형용사인가요?,4
4,화나다 / 차이다,4


In [15]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data,test_size=0.33, random_state=42)

In [16]:
train.to_csv('train.txt', sep="\t")

In [17]:
test.to_csv('test.txt', sep="\t")

In [8]:
dataset_train = nlp.data.TSVDataset("train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("test.txt", field_indices=[1,2], num_discard_samples=1)

In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [16]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## 모델 학습

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [21]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [22]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [23]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [24]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [25]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [26]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 1.7246156930923462 train acc 0.171875
epoch 1 batch id 201 loss 0.119609534740448 train acc 0.7236473880597015

epoch 1 train acc 0.7775887508809021


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9737838649406113


epoch 2 batch id 1 loss 0.32881811261177063 train acc 0.875
epoch 2 batch id 201 loss 0.02608594298362732 train acc 0.9713930348258707

epoch 2 train acc 0.9735068710359408


...


epoch 3 test acc 0.9817913385826772


epoch 4 batch id 1 loss 0.2966510057449341 train acc 0.921875
epoch 4 batch id 201 loss 0.0030922554433345795 train acc 0.9919154228855721

epoch 4 train acc 0.9930353682170543



epoch 4 test acc 0.9812992125984252


epoch 5 batch id 1 loss 0.19872266054153442 train acc 0.953125
epoch 5 batch id 201 loss 0.002110518515110016 train acc 0.9968128109452736

epoch 5 train acc 0.997093023255814



epoch 5 test acc 0.9830216535433071


## 모델 저장

In [28]:
torch.save(model, "/content/first_model.pt")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BERTClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BertModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BertEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local

## 모델 로드

In [85]:
model = torch.load("/content/first_model.pt")


## 추론

In [116]:
sample=pd.Series(["뒷통수 성추행 ㅅㅂ 노래방 올라가는.."])

In [117]:
sample.to_csv('sample2.txt', sep="\t")

In [118]:
dt = nlp.data.TSVDataset("./sample2.txt", field_indices=[0,1], num_discard_samples=1)

In [119]:
datatest = BERTDataset(dt,  1,0, tok, max_len, True, False)

In [120]:
test_dataloader = torch.utils.data.DataLoader(datatest, batch_size=batch_size, num_workers=5)

In [121]:
for (token_ids, valid_length, segment_ids, label) in (tqdm_notebook(test_dataloader)):  
  t= token_ids.long().to(device)
  s = segment_ids.long().to(device)
  v= valid_length

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [122]:
model.forward(t,v,s).argmax()

tensor(4, device='cuda:0')